In [42]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import swifter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [25]:
df = pd.read_csv('train.csv')

In [26]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15932987,ZYNMLE3MV3LK,62728015bec05,1541544490,15,interaction item image,6617798,PT,"Paris, France",desktop,NaN,NaN,NaN
15932988,ZYNMLE3MV3LK,62728015bec05,1541544491,16,clickout item,6617798,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
15932989,ZYNMLE3MV3LK,62728015bec05,1541544540,17,clickout item,2712342,PT,"Paris, France",desktop,Focus on Distance,6617798|1263420|9567886|1161323|149768|1890735...,58|96|55|75|90|60|233|104|150|145|328|207|150|...
15932990,ZYNMLE3MV3LK,62728015bec05,1541544967,18,change of sort order,interaction sort button,PT,"Paris, France",desktop,NaN,NaN,NaN


In [4]:
# import math
# import pandas as pd
# import numpy as np


# GR_COLS = ["user_id", "session_id", "timestamp", "step"]


# def get_submission_target(df):
#     """Identify target rows with missing click outs."""

#     mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
#     df_out = df[mask]

#     return df_out


# def get_popularity(df):
#     """Get number of clicks that each item received in the df."""

#     mask = df["action_type"] == "clickout item"
#     df_clicks = df[mask]
#     df_item_clicks = (
#         df_clicks
#         .groupby("reference")
#         .size()
#         .reset_index(name="n_clicks")
#         .transform(lambda x: x.astype(int))
#     )

#     return df_item_clicks


# def string_to_array(s):
#     """Convert pipe separated string to array."""

#     if isinstance(s, str):
#         out = s.split("|")
#     elif math.isnan(s):
#         out = []
#     else:
#         raise ValueError("Value must be either string of nan")
#     return out


# def explode(df_in, col_expl):
#     """Explode column col_expl of array type into multiple rows."""

#     df = df_in.copy()
#     df.loc[:, col_expl] = df[col_expl].apply(string_to_array)

#     df_out = pd.DataFrame(
#         {col: np.repeat(df[col].values,
#                         df[col_expl].str.len())
#          for col in df.columns.drop(col_expl)}
#     )

#     df_out.loc[:, col_expl] = np.concatenate(df[col_expl].values)
#     df_out.loc[:, col_expl] = df_out[col_expl].apply(int)

#     return df_out


# def group_concat(df, gr_cols, col_concat):
#     """Concatenate multiple rows into one."""

#     df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out


def calc_recommendation(df_expl, df_pop):
    """Calculate recommendations based on popularity of items.

    The final data frame will have an impression list sorted according to the number of clicks per item in a reference data frame.

    :param df_expl: Data frame with exploded impression list
    :param df_pop: Data frame with items and number of clicks
    :return: Data frame with sorted impression list according to popularity in df_pop
    """

    df_expl_clicks = (
        df_expl[GR_COLS + ["impressions"]]
        .merge(df_pop,
               left_on="impressions",
               right_on="reference",
               how="left")
    )

    df_out = (
        df_expl_clicks
        .assign(impressions=lambda x: x["impressions"].apply(str))
        .sort_values(GR_COLS + ["n_clicks"],
                     ascending=[True, True, True, True, False])
    )

    df_out = group_concat(df_out, GR_COLS, "impressions")
    df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)

    return df_out

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 73)

In [27]:
cfn = df.current_filters.isnull().sum()

In [28]:
null_current_features = df.shape[0] - cfn

print(null_current_features)

1153112


In [29]:
ni = df.impressions.isnull().sum()

## Defining a function to remove pipes and place elements in a list 

In [30]:
def get_impressions(doc):
    doc = str(doc)
    #doc = re.sub(r'(\|)', ' ', doc)
    doc = doc.split('|')
    return doc

In [31]:
df['sorted'] = df['impressions'].swifter.apply(get_impressions)

In [32]:
sorted_df = df[df['action_type'] == 'clickout item'].drop_duplicates(subset = ['session_id'], keep = 'first')

In [33]:
sorted_df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,sorted
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,"[3400638, 1253714, 3367857, 5100540, 1088584, ..."
115,02SRUT1NQYH1,3599a6f709eab,1541063864,35,clickout item,2795374,FI,"Krakow, Poland",mobile,NaN,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...,"[2795374, 5582964, 1088390, 2781070, 1258068, ..."
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Res...,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...,"[1306936, 56482, 2842358, 6881276, 65685, 6325..."
180,0473FZ8UNXRS,bcc452f3350eb,1541062532,3,clickout item,3143258,AU,"Legian, Indonesia",desktop,NaN,1258184|3866722|8929970|2315702|116619|1511641...,51|43|69|49|62|50|55|42|87|46|43|114|194|50|19...,"[1258184, 3866722, 8929970, 2315702, 116619, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15932943,ZRWO358OG359,92b7ab1287edf,1541489523,5,clickout item,207081,FR,"Bordeaux, France",desktop,NaN,3133696|50220|207081|50584|48487|82196|12330|1...,86|93|36|72|67|79|156|66|75|72|86|35|96|86|117...,"[3133696, 50220, 207081, 50584, 48487, 82196, ..."
15932965,ZU702MR210D2,6b66cb0cfb518,1541498306,10,clickout item,3811156,JP,"Asahikawa, Japan",mobile,NaN,3811156|2337580|8787688|9920906|1553743|462814...,55|35|51|59|54|63|26|43|52|20|52|31|42|51|50|5...,"[3811156, 2337580, 8787688, 9920906, 1553743, ..."
15932966,ZV6EJXAD9929,f701be9e58e9a,1541532352,1,clickout item,9771628,FR,"New York, USA",mobile,NaN,9771628|5105492|8682|1372901|1321682|1364388|3...,281|183|281|266|238|263|324|345|466|179|186|24...,"[9771628, 5105492, 8682, 1372901, 1321682, 136..."
15932970,ZVIDWWE0KWNB,e7916050980d9,1541532069,4,clickout item,8985292,BR,"New York, USA",desktop,NaN,8640|8816|8791|2298566|1811137|8708|67445|8825...,215|173|188|258|199|133|309|243|211|178|178|18...,"[8640, 8816, 8791, 2298566, 1811137, 8708, 674..."


In [34]:
sorted_df.city.value_counts()

London, United Kingdom          12625
Tokyo, Japan                     9475
New York, USA                    8514
Paris, France                    8381
Las Vegas, USA                   7255
                                ...  
Südlohn, Germany                    1
Hoogkarspel, Netherlands            1
Mnichovice, Czech Republic          1
Saint-Bon-Tarentaise, France        1
Skærbæk, Denmark                    1
Name: city, Length: 29625, dtype: int64

In [35]:
london_df = sorted_df[sorted_df['city'] == 'London, United Kingdom']

In [36]:
london_df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,sorted
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
4950,AL1GXKFSX17S,b8298d3a5830d,1541108234,27,clickout item,12695,AT,"London, United Kingdom",desktop,NaN,1819741|46317|47115|1670405|12695|47672|46574|...,214|186|208|259|213|203|130|156|195|222|234|21...,"[1819741, 46317, 47115, 1670405, 12695, 47672,..."
5926,DKEJ7CIFCAJ9,069e185e3c8b3,1541080069,6,clickout item,13845,HK,"London, United Kingdom",mobile,NaN,13845|5722432|149078|1670405|12710|12763|13968...,105|115|171|166|127|152|590|132|85|92|43|71|13...,"[13845, 5722432, 149078, 1670405, 12710, 12763..."
5954,DPC398J0NG7A,9d6bd4ac6386b,1541033963,13,clickout item,2195678,FR,"London, United Kingdom",mobile,NaN,2195678|12681|46468|926625|46535|1160364|93817...,235|282|377|309|913|2067|242|601|255|259|295|3...,"[2195678, 12681, 46468, 926625, 46535, 1160364..."
6432,FBRAKWKY8KK6,2d2c12c926cf3,1541085283,12,clickout item,664366,UK,"London, United Kingdom",mobile,NaN,664366|6728838|3366712|920457|18456|2097720|26...,112|74|92|118|163|192|67|144|139|93|310|146|84...,"[664366, 6728838, 3366712, 920457, 18456, 2097..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15928395,LVTHS2MARKGO,ed08f69e481a5,1541515959,1,clickout item,98752,IT,"London, United Kingdom",mobile,NaN,46571|81664|1412769|98752|47155|48099|1073868|...,844|235|532|249|343|228|144|250|371|208|481|26...,"[46571, 81664, 1412769, 98752, 47155, 48099, 1..."
15929965,R0H3JB8AO40L,f97fd1cd3f9da,1541536223,1,clickout item,3755574,UK,"London, United Kingdom",mobile,NaN,3755574|6696124|5833050|7809234|46884|18439|12...,396|585|117|420|406|441|494|526|347|343|148|35...,"[3755574, 6696124, 5833050, 7809234, 46884, 18..."
15930734,SW9S5F3MILDI,2ccc70c40055f,1541535137,2,clickout item,6437326,US,"London, United Kingdom",mobile,NaN,894693|46408|81700|47433|81671|152883|46930|94...,104|74|92|107|57|136|151|99|60|45|63|55|305|11...,"[894693, 46408, 81700, 47433, 81671, 152883, 4..."
15930746,SXGJ96Z0I8D0,c89fc3eaf38fa,1541537924,12,clickout item,48161,UK,"London, United Kingdom",tablet,Sort by Price|5 Star,2627696|46547|48071|2489180|47075|47136|47717|...,413|451|494|503|517|656|678|725|755|828|877|11...,"[2627696, 46547, 48071, 2489180, 47075, 47136,..."


In [37]:
london_df['price'] = london_df['prices'].swifter.apply(get_impressions)

//anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
london_df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,sorted,price
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...","[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ..."
4950,AL1GXKFSX17S,b8298d3a5830d,1541108234,27,clickout item,12695,AT,"London, United Kingdom",desktop,NaN,1819741|46317|47115|1670405|12695|47672|46574|...,214|186|208|259|213|203|130|156|195|222|234|21...,"[1819741, 46317, 47115, 1670405, 12695, 47672,...","[214, 186, 208, 259, 213, 203, 130, 156, 195, ..."
5926,DKEJ7CIFCAJ9,069e185e3c8b3,1541080069,6,clickout item,13845,HK,"London, United Kingdom",mobile,NaN,13845|5722432|149078|1670405|12710|12763|13968...,105|115|171|166|127|152|590|132|85|92|43|71|13...,"[13845, 5722432, 149078, 1670405, 12710, 12763...","[105, 115, 171, 166, 127, 152, 590, 132, 85, 9..."
5954,DPC398J0NG7A,9d6bd4ac6386b,1541033963,13,clickout item,2195678,FR,"London, United Kingdom",mobile,NaN,2195678|12681|46468|926625|46535|1160364|93817...,235|282|377|309|913|2067|242|601|255|259|295|3...,"[2195678, 12681, 46468, 926625, 46535, 1160364...","[235, 282, 377, 309, 913, 2067, 242, 601, 255,..."
6432,FBRAKWKY8KK6,2d2c12c926cf3,1541085283,12,clickout item,664366,UK,"London, United Kingdom",mobile,NaN,664366|6728838|3366712|920457|18456|2097720|26...,112|74|92|118|163|192|67|144|139|93|310|146|84...,"[664366, 6728838, 3366712, 920457, 18456, 2097...","[112, 74, 92, 118, 163, 192, 67, 144, 139, 93,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15928395,LVTHS2MARKGO,ed08f69e481a5,1541515959,1,clickout item,98752,IT,"London, United Kingdom",mobile,NaN,46571|81664|1412769|98752|47155|48099|1073868|...,844|235|532|249|343|228|144|250|371|208|481|26...,"[46571, 81664, 1412769, 98752, 47155, 48099, 1...","[844, 235, 532, 249, 343, 228, 144, 250, 371, ..."
15929965,R0H3JB8AO40L,f97fd1cd3f9da,1541536223,1,clickout item,3755574,UK,"London, United Kingdom",mobile,NaN,3755574|6696124|5833050|7809234|46884|18439|12...,396|585|117|420|406|441|494|526|347|343|148|35...,"[3755574, 6696124, 5833050, 7809234, 46884, 18...","[396, 585, 117, 420, 406, 441, 494, 526, 347, ..."
15930734,SW9S5F3MILDI,2ccc70c40055f,1541535137,2,clickout item,6437326,US,"London, United Kingdom",mobile,NaN,894693|46408|81700|47433|81671|152883|46930|94...,104|74|92|107|57|136|151|99|60|45|63|55|305|11...,"[894693, 46408, 81700, 47433, 81671, 152883, 4...","[104, 74, 92, 107, 57, 136, 151, 99, 60, 45, 6..."
15930746,SXGJ96Z0I8D0,c89fc3eaf38fa,1541537924,12,clickout item,48161,UK,"London, United Kingdom",tablet,Sort by Price|5 Star,2627696|46547|48071|2489180|47075|47136|47717|...,413|451|494|503|517|656|678|725|755|828|877|11...,"[2627696, 46547, 48071, 2489180, 47075, 47136,...","[413, 451, 494, 503, 517, 656, 678, 725, 755, ..."


In [39]:
expdfprice = london_df.explode('price')

In [40]:
expdfprice

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,sorted,price
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",104
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",92
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",100
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",103
121,03K8AXBL4BX2,ec139e10b9238,1541100322,6,clickout item,1032816,UK,"London, United Kingdom",desktop,NaN,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,12669|1160332|47617|6255610|46875|47412|116036...,51|314|147|143|280|102|701|229|81|62|759|120|6...,"[12669, 1160332, 47617, 6255610, 46875, 47412,...",447
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,12669|1160332|47617|6255610|46875|47412|116036...,51|314|147|143|280|102|701|229|81|62|759|120|6...,"[12669, 1160332, 47617, 6255610, 46875, 47412,...",366
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,12669|1160332|47617|6255610|46875|47412|116036...,51|314|147|143|280|102|701|229|81|62|759|120|6...,"[12669, 1160332, 47617, 6255610, 46875, 47412,...",531
15932569,YH5BH5INZMHX,068dcaa3b08b4,1541518492,1,clickout item,1160332,UK,"London, United Kingdom",desktop,NaN,12669|1160332|47617|6255610|46875|47412|116036...,51|314|147|143|280|102|701|229|81|62|759|120|6...,"[12669, 1160332, 47617, 6255610, 46875, 47412,...",90
